In [1]:
# ==============================================================================
# SVR (epsilon & nu, RBF)
# ==============================================================================

import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product


# --- 1) Path setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()


NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


# --- 2) Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,  # expanding
    EWMA_CORR_SPEC,  # ewma
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.SVR import ForecastModel  # SVR model wrapper


# --- 3) Config ---
USE_DYNAMIC_FI_PIPELINE = True  # False = standard setups (I & II)
SEED = 42

MODEL_NAME = "svr_dynamic_fi" if USE_DYNAMIC_FI_PIPELINE else "svr_with_target"

outputs_for_model(MODEL_NAME)
print(f"--- Starting tuning for: {MODEL_NAME} ---")


# --- 4) Load data ---
y = load_target()
X_ifo = load_ifo_features()

# Align indices
idx_common = y.index.intersection(X_ifo.index)
y = y.loc[idx_common]
X_ifo = X_ifo.loc[idx_common]

X_full_lagged = None
rolling_imp = None
y_fi = None

if USE_DYNAMIC_FI_PIPELINE:
    FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp = load_rolling_importance(base_dir=FI_BASE_DIR)

        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp = rolling_imp.loc[idx_fi]
        print(f"Dynamic FI mode: loaded {X_full_lagged.shape[1]} features.")
    except FileNotFoundError:
        print("ERROR: Dynamic FI artifacts not found.")
        sys.exit(1)
else:
    print(f"Full FE mode (Setup I/II): {X_ifo.shape[1]} base features.")


# --- 5) Default config (thesis policy) ---
def get_thesis_cfg() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    cfg.policy_window = 24
    cfg.policy_decay = 0.97
    cfg.selection_mode = "decayed_best"
    return cfg


cfg_obj = get_thesis_cfg()


# --- 6) Grid definition ---
def build_grid_full_fe():
    """Setup I (ifo) and Setup II (ifo + target blocks)."""

    # A) FE & DR (thesis grid)
    lag_candidates = [tuple(range(7))]

    corr_opts = [
        {"corr_spec": dict(DEFAULT_CORR_SPEC)},
        {"corr_spec": dict(EWMA_CORR_SPEC)},
    ]

    k1_opts = [700]
    red_opts = [0.9, 1.0]

    dr_opts = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_kmax": 30, "pca_var_target": 0.99},
        {"dr_method": "pls", "pls_components": 30},
    ]

    # B) Setup II (target blocks)
    block_opts = [
        None,  # Setup I
        # ["AR1", "Chronos", "TSFresh"]  # Setup IIb
    ]

    # C) Weights
    weight_opts = [
        {"sample_weight_decay": None},
        {"sample_weight_decay": 0.99},
    ]

    # D) SVR hyperparams
    kernel_list = ["rbf"]
    C_list = [0.1, 1.0, 10.0, 100.0]
    gamma_list = ["scale", 0.01, 0.1]
    epsilon_list = [0.01, 0.1, 0.3]
    nu_list = [0.1, 0.3]
    variant_list = ["epsilon", "nu"]

    shrinking_list = [True]
    max_iter_list = [-1]  # unlimited (until tol)

    grid = []

    # 1) FE loop
    for lags, corr, k1, red, dr in product(lag_candidates, corr_opts, k1_opts, red_opts, dr_opts):
        base_fe = {
            "lag_candidates": lags,
            "k1_topk": k1,
            "redundancy_param": red,
            **dr,
            **corr,
        }

        # 2) Blocks & weights
        for blocks, weights in product(block_opts, weight_opts):

            # 3) Model HPs: epsilon-SVR and nu-SVR
            for variant in variant_list:
                if variant == "epsilon":
                    for kern, C, gam, eps, shrink, m_iter in product(
                        kernel_list, C_list, gamma_list, epsilon_list, shrinking_list, max_iter_list
                    ):
                        hp = {
                            **base_fe,
                            "target_block_set": blocks,
                            **weights,
                            "variant": "epsilon",
                            "kernel": kern,
                            "C": C,
                            "gamma": gam,
                            "epsilon": eps,
                            "shrinking": shrink,
                            "max_iter": m_iter,
                            "tol": 1e-3,
                            "seed": SEED,
                        }
                        grid.append(hp)
                else:  # nu-SVR
                    for kern, C, gam, nu, shrink, m_iter in product(
                        kernel_list, C_list, gamma_list, nu_list, shrinking_list, max_iter_list
                    ):
                        hp = {
                            **base_fe,
                            "target_block_set": blocks,
                            **weights,
                            "variant": "nu",
                            "kernel": kern,
                            "C": C,
                            "gamma": gam,
                            "nu": nu,
                            "shrinking": shrink,
                            "max_iter": m_iter,
                            "tol": 1e-3,
                            "seed": SEED,
                        }
                        grid.append(hp)

    return grid


def build_grid_dynamic_fi():
    """Setup III: dynamic feature importance via strict top-N."""

    n_features_list = [50]  # fixed: 50 features
    weight_opts = [{"sample_weight_decay": None}, {"sample_weight_decay": 0.99}]

    kernel_list = ["rbf"]
    C_list = [0.1, 1.0, 10.0, 100.0]
    gamma_list = ["scale", 0.01, 0.1]
    epsilon_list = [0.01, 0.1, 0.3]
    nu_list = [0.1, 0.3]
    variant_list = ["epsilon", "nu"]
    shrinking_list = [True]
    max_iter_list = [-1]

    grid = []
    for n_feat, w in product(n_features_list, weight_opts):
        for variant in variant_list:
            if variant == "epsilon":
                for kern, C, gam, eps, shrink, m_iter in product(
                    kernel_list, C_list, gamma_list, epsilon_list, shrinking_list, max_iter_list
                ):
                    hp = {
                        "n_features_to_use": n_feat,
                        **w,
                        "variant": "epsilon",
                        "kernel": kern,
                        "C": C,
                        "gamma": gam,
                        "epsilon": eps,
                        "shrinking": shrink,
                        "max_iter": m_iter,
                        "tol": 1e-3,
                        "seed": SEED,
                    }
                    grid.append(hp)
            else:
                for kern, C, gam, nu, shrink, m_iter in product(
                    kernel_list, C_list, gamma_list, nu_list, shrinking_list, max_iter_list
                ):
                    hp = {
                        "n_features_to_use": n_feat,
                        **w,
                        "variant": "nu",
                        "kernel": kern,
                        "C": C,
                        "gamma": gam,
                        "nu": nu,
                        "shrinking": shrink,
                        "max_iter": m_iter,
                        "tol": 1e-3,
                        "seed": SEED,
                    }
                    grid.append(hp)
    return grid


# --- 7) Run ---
if USE_DYNAMIC_FI_PIPELINE:
    grid = build_grid_dynamic_fi()
    print(f"Dynamic FI grid size (Setup III): {len(grid)} configurations.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,  # dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp,
        keep_top_k_final=5,
        min_survivors_per_block=5,
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,  # dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp,
    )

else:
    grid = build_grid_full_fe()
    print(f"Full FE grid size (Setup I & II): {len(grid)} configurations.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
        keep_top_k_final=5,
        min_survivors_per_block=5,
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
    )

print("\nTuning finished.")



--- Starte Tuning für: svr_dynamic_fi ---
INFO in load_ifo_features: Renaming columns to ensure validity.
Dynamic FI Modus: 4320 Features geladen.
Dynamic FI Grid Größe (Setup III): 120 Konfigurationen.
[Stage A] Using DYNAMIC FI (Gleis 3) pipeline.
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=120
  - Config 1/120
    · Month 5/20 processed | running...RMSE=1.6554
    · Month 10/20 processed | running...RMSE=1.3364
    · Month 15/20 processed | running...RMSE=1.2579
    · Month 20/20 processed | running...RMSE=1.1327
  - Config 2/120
    · Month 5/20 processed | running...RMSE=1.6539
    · Month 10/20 processed | running...RMSE=1.3341
    · Month 15/20 processed | running...RMSE=1.2545
    · Month 20/20 processed | running...RMSE=1.1271
  - Config 3/120
    · Month 5/20 processed | running...RMSE=1.6412
    · Month 10/20 processed | running...RMSE=1.3197
    · Month 15/20 processed | running...RMSE=1.2396
    · Month 20/20 processed | running...RMSE=1.1110
  - Config 4/120
 

In [ ]:
323